Given: Data about the movement of empty containers in Hapag-Lloyd's system and need to analyze how efficient these movements are.   

Goal: Identify where the company can improve its operations and use this analysis to create a dashboard for audits.

Hapag-Lloyd moves empty containers to places where they’re needed (deficit locations) from places where they’re not (surplus locations). Sometimes, these movements aren’t efficient, or the proposed movements (actions) don’t actually happen.

Task: Analyze the data to answer the following questions:

Surplus and Demand: Which locations have too many empty containers (surplus), and which need more (deficit)?
Action Success: How many proposed container movements were successfully booked (used) versus canceled (not used)?
Repeated Failures: Are there specific proposals that keep getting canceled but never executed?
Inefficiencies:
Circular Movements: Are containers being moved in unnecessary loops (e.g., back to where they started)?
Indirect Routing: Are containers being sent to an unnecessary middle location before reaching their final destination?

Breakdown:

1. Analyze Surplus and Deficit
Look at where containers are coming from (Subarea From) and where they’re going (Subarea To).
Calculate the total number of containers moved from and to each location.
Find out which locations have extra containers (surplus) and which need containers (deficit).
2. Conversion Rate
Compare proposed container movements (Volume TEU Proposal) with the actual movements (Volume TEU Actual).
Calculate how many proposed movements were successfully completed (conversion rate).
Separate the conversion rates for system-generated actions and user-generated actions.
3. Repeatedly Canceled Proposals
Find proposals (ID Number) that keep getting canceled.
Count how many times a proposal appears with the status “CL” (Canceled).
4. Inefficiencies
Circular Movements: Check if containers of the same type go in a loop (e.g., A → B → C → A) within a short time (4 weeks).
Indirect Routing: Check if containers are sent to an unnecessary stop before reaching their destination. For example:
Surplus: A → B → C
Deficit: A → C (Direct would’ve been better.) or redundancies/inefficiencies like (A → B → C → B) visiting the same location twice in a 4 week window! The timeline is the basis.

In [1]:
#importing the needed libraries

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import zipfile
from icecream import ic
import os
import warnings
import json
from tqdm import tqdm

import logging
from typing import Dict, Tuple
from functools import lru_cache

warnings.filterwarnings('ignore')

### 1. load/read in the data

In [2]:
# --- cell only to be ran once ---

# Path to the zip file
zip_path = r"C:\Users\adeba\OneDrive\Desktop\hapag-lloyd\Case_study_DCT_Analyst_2025.zip" #replace file path

# folder to extract the files
extract_folder = r"C:\Users\adeba\OneDrive\Desktop\hapag-lloyd" #replace file path

# Create the folder if it doesn't exist
# os.makedirs(extract_folder, exist_ok=True)

# Extract files
# with zipfile.ZipFile(zip_path, 'r') as z:
#     z.extractall(extract_folder)
#     ic("Files extracted to:", extract_folder)

In [3]:
# Now read the files directly from the extracted folder
dap_path = os.path.join(extract_folder, "dap.xlsx")
eq_type_path = os.path.join(extract_folder, "eq_type.xlsx")
subarea_path = os.path.join(extract_folder, "subarea.xlsx")

# Load the files into DataFrames
dap_df = pd.read_excel(dap_path)
eq_type_df = pd.read_excel(eq_type_path)
subarea_df = pd.read_excel(subarea_path)

In [5]:
#to have original column names might be needed to reintegrate the data at the end
print('dap_df: ', dap_df.columns, 'eq_type_df: ', eq_type_df.columns, 'subarea_df: ',subarea_df.columns)
original_columns_dap =  dap_df.columns
original_columns_eq = eq_type_df.columns
original_columns_subarea = subarea_df.columns

dap_df:  Index(['ID Number', 'EQ Type', 'Status', 'Source Flag', 'Subarea From',
       'Locode From', 'Subarea To', 'Locode TO', 'Start Yearweek',
       'End Yearweek', 'MoT', 'Volume TEU Proposal', 'Volume TEU Actual'],
      dtype='object') eq_type_df:  Index(['EQ Type', 'EQ Type Desc'], dtype='object') subarea_df:  Index(['Region Code/Name', 'Subregion Code/Name', 'Area Code/Name',
       'Subarea Code/Name', 'UN Locode'],
      dtype='object')


In [ ]:
#quick overview of what I am dealing with
display(dap_df, eq_type_df, subarea_df)

,ID Number,EQ Type,Status,Source Flag,Subarea From,Locode From,Subarea To,Locode TO,Start Yearweek,End Yearweek,MoT,Volume TEU Proposal,Volume TEU Actual
0,7852175,20GE,BK,U,EBNANANR,BEANR,MINCENSA,INNSA,202401,202406,VE,0,50
1,7852174,20GE,BK,U,EBNANANR,BEANR,MINSEMAA,INMAA,202401,202407,VE,0,11
2,7960411,40HC,BK,U,EBIGBSOU,GBSOU,AICVNSGN,VNVUT,202403,202408,VE,0,4
3,7807536,40GE,CL,U,EBIGBLON,GBLGP,ACNSCYTN,CNYTN,202404,202409,VE,0,0
4,7851816,40GE,BK,U,EBIGBSOU,GBSOU,AICVNSGN,VNVUT,202404,202409,VE,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475981,8584842,40HC,BK,U,MASZACPT,ZACPT,LBRPRPNG,BRPNG,202440,202447,MX,0,12
475982,8630606,40RE,BK,U,AJPJPHKT,JPHKT,LPEECGYE,ECGYE,202440,202447,VE,0,40
475983,8603364,40RE,BK,U,AJPJPSMZ,JPSMZ,LPEECGYE,ECGYE,202440,202447,VE,0,4
475984,8627378,20GE,BK,U,LCAGTPBR,GTPBR,LBRRGRIG,BRRIG,202440,202448,MX,0,50


,EQ Type,EQ Type Desc
0,20FL,20' Flat
1,20FL,40' Flat
2,20GE,20' General
3,20HT,20' Hard Top
4,20OT,20' Open Top
5,20RC,20' Reefer Non-foodgrade
6,20RE,20' Reefer
7,20TK,20' Tank
8,20VE,20' Ventilated
9,40FL,40' Flat


,Region Code/Name,Subregion Code/Name,Area Code/Name,Subarea Code/Name,UN Locode
0,S / S. EUROPE,SIB / IBERIA,SIBEM / SP MED,SIBEMBCN / BARCELONA,ADALV
1,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEAUH / ABU DHABI,AEAAN
2,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEAJM / AJMAN,AEAJM
3,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEJEA / JEBEL ALI,AEALQ
4,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEAUH / ABU DHABI,AEAUH
...,...,...,...,...,...
16710,M / M. EAST,MAS / S. AFRICA,MASZA / S. AFRICA,MASZAHRE / HARARE,ZWVFA
16711,M / M. EAST,MAS / S. AFRICA,MASZA / S. AFRICA,MASZAHRE / HARARE,ZWWKI
16712,? / n/a,??? / n/a,????? / n/a,???????? / n/a,NaN
16713,! / n/r,!!! / n/r,!!!!! / n/r,!!!!!!!! / n/r,n/r


### 2. Data Preprocessing and cleaning
#### 2.1. clean column names

In [7]:
def clean_column_names(df):
    # Convert to lowercase and replace spaces with underscores
    df.columns = [x.lower().replace(' ', '_') for x in list(df.columns)]
    return df

In [8]:
# Example usage in preprocessing
dap_cleaned = clean_column_names(dap_df)
eq_type_cleaned = clean_column_names(eq_type_df)
subarea_cleaned = clean_column_names(subarea_df)

#### 2.2 check for duplicates

In [9]:
# Why Check for Duplicate IDs?

"""
The `ID Number` column is described as a unique identifier for action proposals in the dataset. 
This means each ID should correspond to exactly one proposal. 
Duplicate IDs could indicate potential data issues, such as errors in the data ingestion process or 
overlapping proposals. Identifying and addressing these duplicates is critical to ensuring the 
integrity of our analysis.

For example:
- If duplicates exist, it could lead to double-counting proposals in metrics like total container volumes.
- Erroneous duplicates might mask inefficiencies in the DAP system or skew decision-making insights.

Therefore, checking for duplicate `ID Number`s is a necessary quality assurance step in preparing the data for analysis.
"""

# Code: Check for Duplicate IDs
duplicate_ids = dap_cleaned[dap_cleaned.duplicated(subset=["id_number"])]
print(f"Number of duplicate IDs: {duplicate_ids.shape[0]}")

if not duplicate_ids.empty:
    print("Duplicate IDs found! Displaying the rows for investigation:")
    display(duplicate_ids)
else:
    print("No duplicate IDs found. The dataset is clean in this regard.")


Number of duplicate IDs: 230266
Duplicate IDs found! Displaying the rows for investigation:


,id_number,eq_type,status,source_flag,subarea_from,locode_from,subarea_to,locode_to,start_yearweek,end_yearweek,mot,volume_teu_proposal,volume_teu_actual
20090,7790397,20FL,BK,U,ACNSCGGZ,CNHUA,ACNSCSHK,CNSHK,202401,202401,VE,0,3
20091,7792648,20GE,CL,U,ACNSCHKG,HKHKG,ACNSCZSN,CNSUD,202402,202402,VE,0,0
20092,7808128,20GE,BK,U,ACNSCGGZ,CNHUA,ACNSCGGZ,CNGOM,202402,202402,VE,0,50
20093,7792674,20GE,BK,U,AKRRURUS,RUVVO,AKRKRPUS,KRPUS,202401,202402,VE,0,1
20094,7791820,40GE,BK,U,ACNSCZSN,CNXIN,ACNSCZSN,CNJMN,202402,202402,VE,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475980,8664949,20GE,BK,U,LCAHNPCR,HNPCR,LBRRGRIG,BRRIG,202439,202447,MX,0,78
475982,8630606,40RE,BK,U,AJPJPHKT,JPHKT,LPEECGYE,ECGYE,202440,202447,VE,0,40
475983,8603364,40RE,BK,U,AJPJPSMZ,JPSMZ,LPEECGYE,ECGYE,202440,202447,VE,0,4
475984,8627378,20GE,BK,U,LCAGTPBR,GTPBR,LBRRGRIG,BRRIG,202440,202448,MX,0,50


#### 2.3 check for exact duplicates

In [10]:
def check_exact_duplicates(df, unique_id_column):
    """
    Function to check for duplicates based on a unique ID column and verify identical values across all columns.
    
    Parameters:
    - df: DataFrame to check for duplicates
    - unique_id_column: The column that should uniquely identify each row
    
    Returns:
    - duplicates_with_discrepancies: Rows where the same ID has different values in other columns
    - exact_duplicates: True duplicates with identical values in all columns
    """
    # Identify duplicate IDs
    duplicate_rows = df[df.duplicated(subset=[unique_id_column], keep=False)]
    
    # Group duplicates by the unique ID and check for discrepancies
    # Efficient comparison by transforming the group into a single sorted string for each group
    grouped = duplicate_rows.groupby(unique_id_column)
    discrepancies = grouped.filter(
        lambda group: len(group.drop_duplicates()) > 1
    )
    
    # Exact duplicates can be determined once
    exact_duplicates = duplicate_rows.drop_duplicates()
    
    return discrepancies, exact_duplicates

In [11]:
# checking for discrepancies in the duplicates, rows must match 1:1
duplicates_with_discrepancies, exact_duplicates = check_exact_duplicates(dap_cleaned, unique_id_column="id_number")

# Display results
if not duplicates_with_discrepancies.empty:
    print("Duplicates with discrepancies found! These need further investigation:")
    display(duplicates_with_discrepancies)
else:
    print("No duplicates with discrepancies found. All duplicates are exact matches.")

print(f"Number of exact duplicates: {exact_duplicates.shape[0]}")

No duplicates with discrepancies found. All duplicates are exact matches.
Number of exact duplicates: 230266


#### 2.4 remove (exact) duplicates

In [12]:
# can now safely drop duplicates
dap_cleaned = dap_cleaned.drop_duplicates()
dap_cleaned

,id_number,eq_type,status,source_flag,subarea_from,locode_from,subarea_to,locode_to,start_yearweek,end_yearweek,mot,volume_teu_proposal,volume_teu_actual
0,7852175,20GE,BK,U,EBNANANR,BEANR,MINCENSA,INNSA,202401,202406,VE,0,50
1,7852174,20GE,BK,U,EBNANANR,BEANR,MINSEMAA,INMAA,202401,202407,VE,0,11
2,7960411,40HC,BK,U,EBIGBSOU,GBSOU,AICVNSGN,VNVUT,202403,202408,VE,0,4
3,7807536,40GE,CL,U,EBIGBLON,GBLGP,ACNSCYTN,CNYTN,202404,202409,VE,0,0
4,7851816,40GE,BK,U,EBIGBSOU,GBSOU,AICVNSGN,VNVUT,202404,202409,VE,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475966,8601399,20OT,CL,U,MASZADUR,ZADUR,LBRSCITJ,BRITJ,202440,202446,MX,0,0
475967,8601455,20OT,CL,U,MASZADUR,ZADUR,LBRSCITJ,BRITJ,202440,202446,MX,0,0
475968,8568371,20OT,BK,U,MASZACPT,ZACPT,LBRSCITJ,BRITJ,202440,202446,MX,0,2
475978,8575159,40HC,BK,U,MASZACPT,ZACPT,LBRPRPNG,BRPNG,202440,202446,MX,0,36


In [13]:
subarea_df

,region_code/name,subregion_code/name,area_code/name,subarea_code/name,un_locode
0,S / S. EUROPE,SIB / IBERIA,SIBEM / SP MED,SIBEMBCN / BARCELONA,ADALV
1,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEAUH / ABU DHABI,AEAAN
2,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEAJM / AJMAN,AEAJM
3,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEJEA / JEBEL ALI,AEALQ
4,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEAUH / ABU DHABI,AEAUH
...,...,...,...,...,...
16710,M / M. EAST,MAS / S. AFRICA,MASZA / S. AFRICA,MASZAHRE / HARARE,ZWVFA
16711,M / M. EAST,MAS / S. AFRICA,MASZA / S. AFRICA,MASZAHRE / HARARE,ZWWKI
16712,? / n/a,??? / n/a,????? / n/a,???????? / n/a,NaN
16713,! / n/r,!!! / n/r,!!!!! / n/r,!!!!!!!! / n/r,n/r


### 3. Data Validation
#### 3.1 Validate Locations in the DAP data

In [14]:
#check if a locode can have more than one sobarea parent
def check_locode_multiple_subareas(df):
       # Count unique subarea codes per UN locode
   locode_analysis = df.groupby('un_locode')['subarea_code/name'].agg(['nunique', 'unique'])
   
   # Filter for UN locodes with multiple subareas
   multiple_subareas = locode_analysis[locode_analysis['nunique'] > 1]
   
   if len(multiple_subareas) > 0:
       print(f"Found {len(multiple_subareas)} UN locodes with multiple subareas:")
       for locode, row in multiple_subareas.iterrows():
           print(f"\nUN Locode: {locode}")
           print(f"Subareas: {row['unique']}")
   else:
       print("No UN locodes have multiple subarea parents")
       
   return multiple_subareas

# Run analysis
duplicates = check_locode_multiple_subareas(subarea_df)

No UN locodes have multiple subarea parents


In [15]:
def validate_locations(dap_df, subarea_df):
       # Extract unique values and master data
   dap_subareas_from = set(dap_df['subarea_from'].unique())
   dap_subareas_to = set(dap_df['subarea_to'].unique())
   dap_locodes_from = set(dap_df['locode_from'].unique())
   dap_locodes_to = set(dap_df['locode_to'].unique())

   master_subareas = set(subarea_df['subarea_code/name'].str.split('/').str[0].str.strip().unique())
   master_locodes = set(subarea_df['un_locode'].str.strip().unique())

   # Find invalid values
   invalid = {
       'invalid_subareas_from': dap_subareas_from - master_subareas,
       'invalid_subareas_to': dap_subareas_to - master_subareas,
       'invalid_locodes_from': dap_locodes_from - master_locodes,
       'invalid_locodes_to': dap_locodes_to - master_locodes
   }

   # Create invalid data frame from original data
   invalid_dap = dap_df[
       (dap_df['subarea_from'].isin(invalid['invalid_subareas_from'])) |
       (dap_df['subarea_to'].isin(invalid['invalid_subareas_to'])) |
       (dap_df['locode_from'].isin(invalid['invalid_locodes_from'])) |
       (dap_df['locode_to'].isin(invalid['invalid_locodes_to']))
   ]

   # Create valid data frame
   valid_dap = dap_df[
       ~(dap_df['subarea_from'].isin(invalid['invalid_subareas_from'])) &
       ~(dap_df['subarea_to'].isin(invalid['invalid_subareas_to'])) &
       ~(dap_df['locode_from'].isin(invalid['invalid_locodes_from'])) &
       ~(dap_df['locode_to'].isin(invalid['invalid_locodes_to']))
   ]

   return invalid, invalid_dap, valid_dap



In [16]:
# extract rows with unindentified location (not present in the subarea_df) from the dap data
invalid_values, unidentified_locations, dap_with_location = validate_locations(dap_cleaned, subarea_cleaned)
print(f"Total rows: {len(dap_cleaned)}")
print(f"Invalid rows: {len(unidentified_locations)}")
print(f"Valid rows: {len(dap_with_location)}")

Total rows: 245720
Invalid rows: 3621
Valid rows: 242099


In [17]:
#check for duplicates in the new dap data and the unidentified locations dataframe
def check_duplicates(dap_with_location, unidentified_locations):
       # Check duplicates in valid locations
   valid_dupes = dap_with_location[dap_with_location.duplicated(subset=['id_number'], keep=False)]
   
   # Check duplicates in invalid locations
   invalid_dupes = unidentified_locations[unidentified_locations.duplicated(subset=['id_number'], keep=False)]
   
   print(f"Duplicates in valid locations: {len(valid_dupes)}")
   print(f"Duplicates in invalid locations: {len(invalid_dupes)}")
   
   return valid_dupes, invalid_dupes

valid_duplicates, invalid_duplicates = check_duplicates(dap_with_location, unidentified_locations)

Duplicates in valid locations: 0
Duplicates in invalid locations: 0


In [18]:
unidentified_locations

,id_number,eq_type,status,source_flag,subarea_from,locode_from,subarea_to,locode_to,start_yearweek,end_yearweek,mot,volume_teu_proposal,volume_teu_actual
83,7828978,40FL,CL,U,EBN,NaN,ENOFIHEL,FIHEL,202403,202404,NaN,16,0
111,7801745,20GE,CL,U,EGE,NaN,ENOFIKOK,FIKOK,202402,202403,NaN,30,0
144,7769091,40HC,CL,U,EBN,NaN,EGEFFFRA,NaN,202402,202403,NaN,40,0
145,7774708,40GE,CL,U,EBN,NaN,EGCFFFRA,DEFRA,202403,202404,NaN,40,0
146,7791908,40HC,CL,U,EBN,NaN,EGCSTSTR,DEWOE,202403,202404,NaN,80,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
466472,8538791,40FL,CL,U,M,NaN,EBNRTRTM,NLRTM,202438,202440,NaN,30,0
466474,8538805,40FL,CL,U,M,NaN,EBNRTRTM,NLRTM,202439,202441,NaN,30,0
466475,8538191,40OT,CL,U,M,NaN,EBNRTRTM,NLRTM,202439,202441,NaN,30,0
466476,8549254,40FL,CL,U,M,NaN,EBNANANR,BEANR,202438,202440,NaN,36,0


In [21]:
#from eyeballing the dataframe, it seems unidentified locations data has the source_flag 'U'
ic(unidentified_locations['status'].unique())
ic(unidentified_locations['source_flag'].unique())
ic(unidentified_locations['volume_teu_actual'].unique())

#conclusion, unidentified locations data were all user generated. WHY?

ic| unidentified_locations['status'].unique(): array(['CL', 'BK'], dtype=object)
ic| unidentified_locations['source_flag'].unique(): array(['U'], dtype=object)
ic| unidentified_locations['volume_teu_actual'].unique(): array([  0,   5,   2, 100,  56,   8,  48, 200,   1,  50, 140, 112,  60,
                                                                  34,  26,  46,   6,  40,  80,  68, 116, 120,   3,   4,  18,  12])


array([  0,   5,   2, 100,  56,   8,  48, 200,   1,  50, 140, 112,  60,
        34,  26,  46,   6,  40,  80,  68, 116, 120,   3,   4,  18,  12])

In [22]:
#digging deeper with the unidentified locations
unidentified_locations[unidentified_locations[['locode_from', 'locode_to']].isna().all(axis=1)]['volume_teu_actual'].unique()
#also situations where unidentified locations dataframe has a missing locode, the Volume TEU Actual is 0!

array([0])

#### 3.2 Validate equipment type in the DAP data

In [19]:
def validate_equipment_types(dap_df, eq_type_df):
       # Extract unique equipment types
   dap_eq_types = set(dap_df['eq_type'].unique())
   master_eq_types = set(eq_type_df['eq_type'].unique())
   
   # Find invalid equipment types
   invalid_eq_types = dap_eq_types - master_eq_types
   
   # Create dataframes for valid and invalid records
   invalid_eq_data = dap_df[dap_df['eq_type'].isin(invalid_eq_types)]
   valid_eq_data = dap_df[~dap_df['eq_type'].isin(invalid_eq_types)]
   
   return {
       'invalid_types': invalid_eq_types,
       'invalid_data': invalid_eq_data,
       'valid_data': valid_eq_data
   }

In [20]:
# check dap equipments in the equipments master data
eq_validation = validate_equipment_types(dap_with_location, eq_type_cleaned)
print(f"Total records: {len(dap_with_location)}")
print(f"Invalid equipment types: {eq_validation['invalid_types']}")
print(f"Records with invalid equipment: {len(eq_validation['invalid_data'])}")
print(f"Records with valid equipment: {len(eq_validation['valid_data'])}")

Total records: 242099
Invalid equipment types: set()
Records with invalid equipment: 0
Records with valid equipment: 242099


#### 3.3 Check for and handle missing values in the DAP data

In [21]:
print(dap_with_location.isnull().sum())

id_number                 0
eq_type                   0
status                    0
source_flag               0
subarea_from              0
locode_from              62
subarea_to                0
locode_to                59
start_yearweek            0
end_yearweek              0
mot                    4286
volume_teu_proposal       0
volume_teu_actual         0
dtype: int64


In [22]:
display(dap_with_location[dap_with_location['locode_from'].isna()], dap_with_location[dap_with_location['locode_to'].isna()], dap_with_location[dap_with_location['mot'].isna()])

,id_number,eq_type,status,source_flag,subarea_from,locode_from,subarea_to,locode_to,start_yearweek,end_yearweek,mot,volume_teu_proposal,volume_teu_actual
9704,7827890,45EQ,CL,U,LCLCUMAR,NaN,LCLCUMAR,NaN,202403,202404,NaN,2,0
9706,7849924,45EQ,CL,U,LCLCUMAR,NaN,LCLCUMAR,NaN,202404,202405,NaN,2,0
9747,7849925,45EQ,CL,U,LCLCUMAR,NaN,LCLCUMAR,NaN,202404,202405,NaN,10,0
47938,7872040,40OT,CL,U,AANAUADL,NaN,ASASGSIN,SGSIN,202405,202407,NaN,20,0
47944,7933760,40HC,CL,U,AANAUMEL,NaN,ASASGSIN,SGSIN,202408,202409,NaN,54,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399853,8537691,20GE,CL,U,ACNNCTAG,NaN,ACNNCTAO,NaN,202436,202437,NaN,500,0
399855,8457537,40HC,CL,U,ACNNCTAG,NaN,ACNNCNGB,NaN,202434,202435,NaN,2000,0
404393,8485728,20GE,CL,U,LCOCOBAQ,NaN,LCOCOCTG,NaN,202435,202435,NaN,50,0
404826,8511621,20OT,CL,U,LCLCUSCU,NaN,LCLCUSCU,NaN,202436,202437,NaN,1,0


,id_number,eq_type,status,source_flag,subarea_from,locode_from,subarea_to,locode_to,start_yearweek,end_yearweek,mot,volume_teu_proposal,volume_teu_actual
9704,7827890,45EQ,CL,U,LCLCUMAR,NaN,LCLCUMAR,NaN,202403,202404,NaN,2,0
9706,7849924,45EQ,CL,U,LCLCUMAR,NaN,LCLCUMAR,NaN,202404,202405,NaN,2,0
9747,7849925,45EQ,CL,U,LCLCUMAR,NaN,LCLCUMAR,NaN,202404,202405,NaN,10,0
48196,7871343,40FL,CL,U,AICVNDAD,NaN,ASASGSIN,NaN,202405,202406,NaN,2,0
52722,7876495,40HC,CL,U,LCADOCAU,NaN,SITITLIV,NaN,202408,202410,NaN,352,0
52723,7880773,40HC,CL,U,LCADOCAU,NaN,SITITGOA,NaN,202407,202410,NaN,496,0
53126,7872469,20GE,CL,U,LCLPAPUQ,NaN,LCLSOLQN,NaN,202406,202407,NaN,1,0
53127,7871669,40RE,CL,U,LCLNOANF,NaN,LCLSOLQN,NaN,202406,202406,NaN,12,0
53134,7871811,40GE,CL,U,LCLNOPAG,NaN,LCLNOANF,NaN,202405,202405,NaN,2,0
56927,7874948,20GE,CL,U,EGCHAHAM,NaN,SITITGOA,NaN,202407,202411,NaN,100,0


,id_number,eq_type,status,source_flag,subarea_from,locode_from,subarea_to,locode_to,start_yearweek,end_yearweek,mot,volume_teu_proposal,volume_teu_actual
14,7756418,40RE,CL,U,EBNRTRTM,NLRTM,MASZACPT,ZACPT,202403,202407,NaN,20,0
21,7808047,40FL,CL,U,EGCHAHAM,DEHAM,ENOFIRAU,FIRAU,202402,202403,NaN,16,0
22,7767837,20GE,CL,U,ENOPLGDY,PLGDY,ENOFIRAU,FIRAU,202401,202402,NaN,30,0
23,7831040,20GE,CL,U,EGCHAHAM,DEHAM,ENOFIOUL,FIOUL,202404,202405,NaN,30,0
24,7828188,40RE,CL,U,EGCHAHAM,DEHAM,ENOPLGDY,PLGDY,202403,202404,NaN,60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471868,8476684,20GE,CL,U,MAWAOLAD,AOLAD,LBRSCITJ,BRITJ,202438,202440,NaN,10,0
471919,8548107,40GE,CL,U,MAWAOLAD,AOLAD,LBRSZSSZ,BRSSZ,202437,202439,NaN,22,0
473477,8552640,40RE,CL,S,MAWAOLAD,AOLAD,LBRPRPNG,BRPNG,202438,202440,NaN,20,0
473692,8552928,40RE,CL,S,MASZADUR,ZADUR,LBRPRPNG,BRPNG,202439,202443,NaN,4,0


In [23]:
dap_with_location[['locode_from', 'locode_to']] = dap_with_location[['locode_from', 'locode_to']].fillna('missing_locode')

In [24]:
dap_with_location[['mot']] = dap_with_location[['mot']].fillna('missing_mot')

#### 3.4 Validate timeline

In [25]:
def is_valid_yearweek(yearweek):
    try:
        year = int(str(yearweek)[:4])
        week = int(str(yearweek)[4:])
        return 1900 <= year <= 2100 and 1 <= week <= 53
    except:
        return False

invalid_starts = dap_with_location[~dap_with_location['start_yearweek'].apply(is_valid_yearweek)]
invalid_ends = dap_with_location[~dap_with_location['end_yearweek'].apply(is_valid_yearweek)]

print("Invalid start yearweeks:", invalid_starts['start_yearweek'].unique())
print("Invalid end yearweeks:", invalid_ends['end_yearweek'].unique())

Invalid start yearweeks: []
Invalid end yearweeks: [303012 320246]


In [26]:
empty_timelines =  [303012, 320246]
dap_with_location[dap_with_location[['start_yearweek', 'end_yearweek']].isin(empty_timelines).any(axis=1)]

,id_number,eq_type,status,source_flag,subarea_from,locode_from,subarea_to,locode_to,start_yearweek,end_yearweek,mot,volume_teu_proposal,volume_teu_actual
158625,8044125,40GE,BK,U,LBRSZSSZ,BRSSZ,LBRSZSSZ,BRSSZ,202413,303012,TR,0,4
289044,8305447,40RE,BK,U,AICVNSGN,VNSGN,AICVNSGN,VNVUT,202427,320246,WW,0,196


In [27]:
dap_with_location = dap_with_location[~dap_with_location[['start_yearweek', 'end_yearweek']].isin(empty_timelines).any(axis=1)]

In [28]:
dap_with_location

,id_number,eq_type,status,source_flag,subarea_from,locode_from,subarea_to,locode_to,start_yearweek,end_yearweek,mot,volume_teu_proposal,volume_teu_actual
0,7852175,20GE,BK,U,EBNANANR,BEANR,MINCENSA,INNSA,202401,202406,VE,0,50
1,7852174,20GE,BK,U,EBNANANR,BEANR,MINSEMAA,INMAA,202401,202407,VE,0,11
2,7960411,40HC,BK,U,EBIGBSOU,GBSOU,AICVNSGN,VNVUT,202403,202408,VE,0,4
3,7807536,40GE,CL,U,EBIGBLON,GBLGP,ACNSCYTN,CNYTN,202404,202409,VE,0,0
4,7851816,40GE,BK,U,EBIGBSOU,GBSOU,AICVNSGN,VNVUT,202404,202409,VE,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475966,8601399,20OT,CL,U,MASZADUR,ZADUR,LBRSCITJ,BRITJ,202440,202446,MX,0,0
475967,8601455,20OT,CL,U,MASZADUR,ZADUR,LBRSCITJ,BRITJ,202440,202446,MX,0,0
475968,8568371,20OT,BK,U,MASZACPT,ZACPT,LBRSCITJ,BRITJ,202440,202446,MX,0,2
475978,8575159,40HC,BK,U,MASZACPT,ZACPT,LBRPRPNG,BRPNG,202440,202446,MX,0,36


In [33]:
# cleaned & valid data
# #dap_with_location.to_csv(r"C:\Users\adeba\Desktop\hapag-lloyd\cleaned_hplld_data.csv")

### 4. Data Analysis

In [29]:
dap_with_location.columns

Index(['id_number', 'eq_type', 'status', 'source_flag', 'subarea_from',
       'locode_from', 'subarea_to', 'locode_to', 'start_yearweek',
       'end_yearweek', 'mot', 'volume_teu_proposal', 'volume_teu_actual'],
      dtype='object')

Todo:
part 1:
1. Flow Analysis:


* Analyze equipment flows between locations
* Calculates net surplus/deficit for each location
* Identifies dominant equipment flows


2. Conversion Rate Analysis:


* Calculates conversion rates (actual/proposed volume) for both system and user-generated actions
* Breaks down performance by source type


3. Inefficiency Analysis:


* Identifies unused actions (proposed but never materialized)

part 2:
* Detects circular movements within 4-week periods
* Finds routing inefficiencies where indirect routes are used instead of available direct routes


#### 4.1 part 1

In [30]:
# Cache the function to improve performance when repeatedly converting yearweek values
@lru_cache(maxsize=1024)


def yearweek_to_date(yearweek: int) -> pd.Timestamp:
    """
    Convert a yearweek integer (e.g., 202401) into a pandas Timestamp.
    
    Args:
        yearweek (int): YearWeek format (YYYYWW).
        
    Returns:
        pd.Timestamp: Corresponding start-of-week datetime.
    """
    year = int(str(yearweek)[:4])  # Extract the year (first 4 digits)
    week = int(str(yearweek)[4:])  # Extract the week number (last 2 digits)
    return pd.to_datetime(f"{year}-W{week:02d}-1", format="%Y-W%W-%w")


def preprocess_dataframe(
    df: pd.DataFrame, min_volume: float, include_cancelled: bool, location_type: str
) -> pd.DataFrame:
    """
    Preprocess the input DataFrame by filtering and adding derived columns.
    
    Args:
        df (pd.DataFrame): Input container movement DataFrame.
        min_volume (float): Minimum volume threshold for filtering.
        include_cancelled (bool): Whether to include cancelled movements.
        location_type (str): Location type ('locode' or 'subarea') to determine columns.

    Returns:
        pd.DataFrame: Preprocessed DataFrame with additional calculated fields.
    """
    # Dynamically determine the location columns based on location_type
    location_from = f"{location_type}_from"
    location_to = f"{location_type}_to"

    # Retain only necessary columns to optimize memory usage and computation
    needed_columns = [
        'status', 'volume_teu_proposal', 'volume_teu_actual',
        'start_yearweek', 'end_yearweek', location_from,
        location_to, 'eq_type', 'source_flag', 'mot'
    ]
    df = df[needed_columns].copy()  # Make a copy of the required columns

    # Apply filtering based on volume and cancellation status
    mask = (df['volume_teu_proposal'] >= min_volume)  # Filter by minimum volume
    if not include_cancelled:
        mask &= (df['status'] != 'CL')  # Exclude cancelled movements if specified
    df = df[mask]

    # Add derived columns for start and end dates based on yearweek
    df['start_date'] = df['start_yearweek'].apply(yearweek_to_date)
    df['end_date'] = df['end_yearweek'].apply(yearweek_to_date)

    # Calculate the duration of each movement in weeks
    df['duration_weeks'] = (df['end_date'] - df['start_date']).dt.days / 7

    return df


def analyze_flows(df: pd.DataFrame, location_type: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Analyze container flows between locations and calculate inflow/outflow stats.
    
    Args:
        df (pd.DataFrame): Preprocessed DataFrame with movement data.
        location_type (str): Location type ('locode' or 'subarea') to determine columns.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: 
            - Aggregated flows between locations.
            - Location-level inflow, outflow, and net flow statistics.
    """
    # Dynamically determine the location columns based on location_type
    location_from = f"{location_type}_from"
    location_to = f"{location_type}_to"

    # Calculate total proposed volume for each flow (grouped by location pair and equipment type)
    flows = df.groupby(
        [location_from, location_to, 'eq_type'], observed=True
    )['volume_teu_proposal'].sum()

    # Calculate inflow, outflow, and net flow at the location level
    location_stats = pd.DataFrame({
        'outflow': df.groupby(location_from, observed=True)['volume_teu_proposal'].sum(),
        'inflow': df.groupby(location_to, observed=True)['volume_teu_proposal'].sum()
    }).fillna(0)  # Fill NaN values with 0 for locations with no inflow or outflow

    # Calculate net flow (inflow - outflow) for each location
    location_stats['net_flow'] = location_stats['inflow'] - location_stats['outflow']

    # Reset index to bring location names into the DataFrame
    location_stats = location_stats.reset_index()

    # Rename columns to meaningful names
    location_stats.columns = ['location', 'outflow', 'inflow', 'net_flow']

    return flows, location_stats


def calculate_conversion(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the conversion rate (actual vs proposed volume) for each source and equipment type.
    
    Args:
        df (pd.DataFrame): Preprocessed DataFrame with movement data.
    
    Returns:
        pd.DataFrame: Aggregated conversion rate for each source and equipment type.
    """
    return df.groupby(['source_flag', 'eq_type'], observed=True).agg({
        'volume_teu_proposal': 'sum',
        'volume_teu_actual': 'sum'
    }).assign(
        conversion_rate=lambda x: (x['volume_teu_actual'] / x['volume_teu_proposal'] * 100).fillna(0)
    )


def find_unused(df: pd.DataFrame, location_type: str) -> pd.DataFrame:
    """
    Identify movements with zero actual volume, aggregating relevant data.
    
    Args:
        df (pd.DataFrame): Input DataFrame containing movement data.
        location_type (str): Location type ('locode' or 'subarea') to determine columns.

    Returns:
        pd.DataFrame: Aggregated data for unused movements.
    """
    # Make a copy of the input DataFrame to avoid unintended side effects
    df = df.copy()

    # Dynamically determine the location columns based on location_type
    location_from = f"{location_type}_from"
    location_to = f"{location_type}_to"

    # Add start and end dates derived from yearweek
    df['start_date'] = df['start_yearweek'].apply(yearweek_to_date)
    df['end_date'] = df['end_yearweek'].apply(yearweek_to_date)

    # Filter for movements with zero actual volume, cancelled status, and positive proposed volume
    mask = (df['volume_teu_actual'] == 0) & (df['status'] == 'CL') & (df['volume_teu_proposal'] > 0)

    # Aggregate data for unused movements
    unused = df[mask].groupby(
        [location_from, location_to, 'eq_type', 'mot'], observed=True
    ).agg({
        'id_number': 'count',
        'volume_teu_proposal': ['sum', 'mean'],
        'start_date': 'min',
        'end_date': 'max'
    }).reset_index()

    # Rename and format aggregated columns for readability
    unused.columns = [
        f"{col[0]}_{col[1]}" if col[1] else col[0]
        for col in unused.columns
    ]
    unused['proposal_timespan_days'] = (
        unused['end_date_max'] - unused['start_date_min']
    ).dt.days
    unused = unused.rename(columns={
        'eq_type': 'equipment_type',
        'mot': 'mode_of_transportation',
        'id_number_count': 'unused_count',
        'volume_teu_proposal_sum': 'total_volume_proposed',
        'volume_teu_proposal_mean': 'avg_volume_proposed',
        'start_date_min': 'first_proposed',
        'end_date_max': 'last_proposed'
    })
    
    # Sort results by unused count and total proposed volume
    return unused.sort_values(
        ['unused_count', 'total_volume_proposed'], ascending=[False, False]
    )

def calculate_metrics(df: pd.DataFrame, location_type: str) -> Dict:
    """
    Calculate various metrics for container movements, including overall stats 
    and breakdowns by equipment type, source flag, and mode of transport.
    
    Args:
        df (pd.DataFrame): Preprocessed DataFrame with movement data.
        location_type (str): Location type ('locode' or 'subarea') to determine columns.

    Returns:
        Dict: A dictionary containing calculated metrics.
    """
    # Dynamically determine the location columns based on location_type
    location_from = f"{location_type}_from"
    location_to = f"{location_type}_to"

    # Aggregate overall metrics for the entire dataset
    overall_metrics = df.agg({
        'volume_teu_proposal': 'sum',  # Total proposed volume
        'volume_teu_actual': 'sum',   # Total actual volume
        'duration_weeks': 'mean'      # Average duration of movements in weeks
    })

    # Calculate the total number of unique routes
    unique_routes = df.groupby(
        [location_from, location_to], observed=True
    ).ngroups  # Count distinct groups of location pairs

    # Compile overall metrics, including conversion rate and total movements
    return {
        'overall': {
            'total_volume_proposed': float(overall_metrics['volume_teu_proposal']),
            'total_volume_actual': float(overall_metrics['volume_teu_actual']),
            'conversion_rate': float(overall_metrics['volume_teu_actual'] / 
                                      overall_metrics['volume_teu_proposal'] * 100 
                                      if overall_metrics['volume_teu_proposal'] > 0 else 0),
            'total_movements': len(df),
            'unique_routes': unique_routes,
            'avg_duration': float(overall_metrics['duration_weeks'])
        },
        # Breakdowns by equipment type, source flag, and mode of transport
        'equipment': df.groupby('eq_type', observed=True).agg({
            'volume_teu_proposal': 'sum',
            'volume_teu_actual': 'sum'
        }).to_dict(),
        'source': df.groupby('source_flag', observed=True).agg({
            'volume_teu_proposal': 'sum',
            'volume_teu_actual': 'sum'
        }).to_dict(),
        'transport': df.groupby('mot', observed=True).agg({
            'volume_teu_proposal': 'sum',
            'volume_teu_actual': 'sum'
        }).to_dict()
    }


def analyze_container_movements(
    df: pd.DataFrame,
    time_window: int = 4,
    min_volume: float = 0,
    include_cancelled: bool = False,
    location_type: str = "locode"
) -> Dict:
    """
    Main function to analyze container movements and produce a comprehensive 
    set of insights, including flow analysis, unused movements, and metrics.
    
    Args:
        df (pd.DataFrame): Input DataFrame containing container movement data.
        time_window (int, optional): Time window for analysis (not used in this implementation).
        min_volume (float, optional): Minimum volume threshold for filtering. Default is 0.
        include_cancelled (bool, optional): Whether to include cancelled movements. Default is False.
        location_type (str, optional): Type of location to analyze ('locode' or 'subarea'). Default is 'locode'.

    Returns:
        Dict: A dictionary containing results of the analysis, including flows, locations,
              conversion rates, unused movements, and overall metrics.
    """
    # Dynamically determine the location columns based on location_type
    location_from = f"{location_type}_from"
    location_to = f"{location_type}_to"

    # Validate that all required columns are present in the input DataFrame
    required_columns = [
        'status', 'volume_teu_proposal', 'volume_teu_actual',
        'start_yearweek', 'end_yearweek',
        location_from, location_to, 'eq_type', 'source_flag', 'mot'
    ]
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")

    # Step 1: Preprocess the data
    processed_df = preprocess_dataframe(df, min_volume, include_cancelled, location_type)
    print("Preprocessing completed.")  # Informational log for tracking progress

    # Step 2: Perform various analyses
    # Analyze flows between locations
    flows, locations = analyze_flows(processed_df, location_type)

    # Calculate conversion rates for different source flags and equipment types
    conversion_rates = calculate_conversion(processed_df)

    # Identify and aggregate data for unused movements
    unused_actions = find_unused(df, location_type)

    # Calculate overall metrics and breakdowns
    metrics = calculate_metrics(processed_df, location_type)

    print("Analysis completed.")  # Informational log for tracking progress

    # Return all analysis results as a dictionary
    return {
        'flows': flows,                  # Aggregated flows between locations
        'locations': locations,          # Inflow, outflow, and net flow stats for locations
        'conversion_rates': conversion_rates,  # Conversion rate metrics
        'unused_actions': unused_actions,      # Unused movements
        'metrics': metrics,              # Overall and breakdown metrics
        'original_df': processed_df      # The preprocessed DataFrame for reference
    }

In [ ]:
# analysis on locode level
results_locode = analyze_container_movements(
    df=dap_with_location,
    time_window=4,
    min_volume=1,
    include_cancelled=False,
    location_type="locode"  # Use "locode_from" and "locode_to"
)

# analysis on subarea level
results_subarea = analyze_container_movements(
    df=dap_with_location,
    time_window=4,
    min_volume=1,
    include_cancelled=False,
    location_type="subarea"  # Use "subarea_from" and "subarea_to"
)


Preprocessing completed.
Analysis completed.
Preprocessing completed.
Analysis completed.


establish regional heirarchies in the output data

In [32]:
#creates explicit and clear regional heirachies in the subarea dataframe
def extract_clean_locations(df):
    """
    Creates new columns with clean location names from code/name columns.
    
    Args:
        df: DataFrame with columns:
            - region_code/name
            - subregion_code/name
            - area_code/name
            - subarea_code/name
            
    Returns:
        DataFrame with four new columns:
            - region
            - subregion
            - area
            - subarea
    """
    def clean_name(value):
        """
        Extracts the name part after '/' and removes any trailing codes.
        
        Args:
            value: String in format 'CODE / NAME'
            
        Returns:
            Clean location name
        """
        # Handle special cases
        if any(x in str(value) for x in ['n/a', 'n/r', 'n/u', '???', '!!!', '***']):
            return ''
            
        try:
            # Split on '/' and take the name part
            name = value.split('/')[1].strip()
            return name
        except:
            return ''
    
    # Create new columns with clean names
    df['region'] = df['region_code/name'].apply(clean_name)
    df['subregion'] = df['subregion_code/name'].apply(clean_name)
    df['area'] = df['area_code/name'].apply(clean_name)
    df['subarea_name'] = df['subarea_code/name'].apply(clean_name)
    
    return df


In [33]:
subarea_cleaned = extract_clean_locations(subarea_cleaned)
subarea_cleaned

,region_code/name,subregion_code/name,area_code/name,subarea_code/name,un_locode,region,subregion,area,subarea_name
0,S / S. EUROPE,SIB / IBERIA,SIBEM / SP MED,SIBEMBCN / BARCELONA,ADALV,S. EUROPE,IBERIA,SP MED,BARCELONA
1,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEAUH / ABU DHABI,AEAAN,M. EAST,ARAB GULF,EMIRATES,ABU DHABI
2,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEAJM / AJMAN,AEAJM,M. EAST,ARAB GULF,EMIRATES,AJMAN
3,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEJEA / JEBEL ALI,AEALQ,M. EAST,ARAB GULF,EMIRATES,JEBEL ALI
4,M / M. EAST,MAG / ARAB GULF,MAGAE / EMIRATES,MAGAEAUH / ABU DHABI,AEAUH,M. EAST,ARAB GULF,EMIRATES,ABU DHABI
...,...,...,...,...,...,...,...,...,...
16710,M / M. EAST,MAS / S. AFRICA,MASZA / S. AFRICA,MASZAHRE / HARARE,ZWVFA,M. EAST,S. AFRICA,S. AFRICA,HARARE
16711,M / M. EAST,MAS / S. AFRICA,MASZA / S. AFRICA,MASZAHRE / HARARE,ZWWKI,M. EAST,S. AFRICA,S. AFRICA,HARARE
16712,? / n/a,??? / n/a,????? / n/a,???????? / n/a,NaN,,,,
16713,! / n/r,!!! / n/r,!!!!! / n/r,!!!!!!!! / n/r,n/r,,,,


In [34]:
#flows
df_subarea_flows = results_subarea['flows'].reset_index()
df_locode_flows = results_locode['flows'].reset_index()

#locations
df_subarea_locations = results_subarea['locations']
df_locode_locations = results_locode['locations']

#conversion_rates
df_subarea_conversion_rates = results_subarea['conversion_rates'].reset_index()
df_locode_conversion_rates = results_locode['conversion_rates'].reset_index()

#unused_actions
df_subarea_unused_actions = results_subarea['unused_actions']
df_locode_unused_actions = results_locode['unused_actions']

#metrics
dict_subarea_metrics = results_subarea['metrics']
dict_locode_metrics = results_locode['metrics']

In [35]:
#function for merging output data with location data
def merge_location_flows(df_flows, subarea_mapping, location_type='locode'):
    """
    Merge flow data with subarea mapping and geographic information, handling multiple data formats.
    
    Parameters:
    -----------
    df_flows : DataFrame
        Can handle different flow data formats:
        1. Single location column with flows:
           - columns: ['location', 'outflow', 'inflow', 'net_flow']
        2. From/To pairs with volume:
           - for locodes: ['locode_from', 'locode_to', 'eq_type', 'volume_teu_proposal']
           - for subareas: ['subarea_from', 'subarea_to', 'eq_type', 'volume_teu_proposal']
    
    subarea_mapping : DataFrame
        Reference mapping with columns:
        - 'un_locode': port/location code
        - 'subarea_code/name': format "CODE / NAME" (e.g., "MAGAEAJM / AJMAN")
        - geography columns: 'region', 'subregion', 'area'
    
    location_type : str
        Specifies the type of location identifiers in df_flows:
        - 'locode': for UN LOCODE format (e.g., 'AEAJM')
        - 'subarea': for subarea code format (e.g., 'MAGAEAJM')
    
    Returns:
    --------
    DataFrame with added geographic information, maintaining original identifiers
    and adding mapped codes where applicable.
    """
    # Create clean mappings
    locode_to_subarea = subarea_mapping[['un_locode', 'subarea_code/name']].drop_duplicates()
    locode_to_subarea['subarea_code'] = locode_to_subarea['subarea_code/name'].str.split(' / ').str[0]
    
    # Create geography mapping
    subarea_code_to_geo = subarea_mapping[['subarea_code/name', 'region', 'subregion', 'area']].drop_duplicates()
    subarea_code_to_geo['subarea_code'] = subarea_code_to_geo['subarea_code/name'].str.split(' / ').str[0]
    subarea_code_to_geo = subarea_code_to_geo.drop('subarea_code/name', axis=1)
    
    # Make a copy to avoid modifying original
    result = df_flows.copy()
    
    # Detect data format based on columns
    has_location = 'location' in result.columns
    has_subarea_pairs = 'subarea_from' in result.columns and 'subarea_to' in result.columns
    has_locode_pairs = 'locode_from' in result.columns and 'locode_to' in result.columns
    
    if has_location:
        # Handle single location column format
        if location_type.lower() == 'locode':
            # Convert locode to subarea code
            result = result.merge(
                locode_to_subarea[['un_locode', 'subarea_code']],
                left_on='location',
                right_on='un_locode',
                how='left'
            ).rename(columns={
                'location': 'locode',
                'subarea_code': 'subarea'
            }).drop('un_locode', axis=1)
            
            # Add geographic information
            result = result.merge(
                subarea_code_to_geo[['subarea_code', 'region', 'subregion', 'area']],
                left_on='subarea',
                right_on='subarea_code',
                how='left'
            ).drop('subarea_code', axis=1)
            
        else:  # subarea type
            result = result.rename(columns={'location': 'subarea'})
            
            # Add geographic information
            result = result.merge(
                subarea_code_to_geo[['subarea_code', 'region', 'subregion', 'area']],
                left_on='subarea',
                right_on='subarea_code',
                how='left'
            ).drop('subarea_code', axis=1)
            
    elif has_subarea_pairs or has_locode_pairs:
        if has_locode_pairs:
            # First merge: origin
            result = result.merge(
                locode_to_subarea[['un_locode', 'subarea_code']],
                left_on='locode_from',
                right_on='un_locode',
                how='left'
            ).rename(columns={'subarea_code': 'subarea_from'}).drop('un_locode', axis=1)
            
            # Second merge: destination
            result = result.merge(
                locode_to_subarea[['un_locode', 'subarea_code']],
                left_on='locode_to',
                right_on='un_locode',
                how='left'
            ).rename(columns={'subarea_code': 'subarea_to'}).drop('un_locode', axis=1)
        
        # Add geographic information for origin
        result = result.merge(
            subarea_code_to_geo[['subarea_code', 'region', 'subregion', 'area']],
            left_on='subarea_from',
            right_on='subarea_code',
            how='left'
        ).drop('subarea_code', axis=1)
        
        # Add geographic information for destination
        result = result.merge(
            subarea_code_to_geo[['subarea_code', 'region', 'subregion', 'area']],
            left_on='subarea_to',
            right_on='subarea_code',
            how='left',
            suffixes=('_from', '_to')
        ).drop('subarea_code', axis=1)
    
    # Drop any remaining unnecessary columns
    cols_to_drop = [col for col in result.columns if col.startswith('subarea_code/name')]
    result = result.drop(cols_to_drop, axis=1)
    
    return result

In [36]:
df_locode_locations

,location,outflow,inflow,net_flow
0,AEAJM,500.0,0.0,-500.0
1,AEAUH,5969.0,13152.0,7183.0
2,AEJEA,52058.0,5657.0,-46401.0
3,AESHJ,1725.0,0.0,-1725.0
4,ALDRZ,651.0,0.0,-651.0
...,...,...,...,...
547,ZACPT,191.0,452.0,261.0
548,ZADUR,1577.0,501.0,-1076.0
549,ZAPLZ,383.0,0.0,-383.0
550,ZAPTN,20.0,0.0,-20.0


In [37]:
#establishing location heirarchies
# For subarea-based flows:
flows_subarea = merge_location_flows(df_subarea_flows, subarea_cleaned, location_type='subarea')
# For locode-based flows:
flows_locode = merge_location_flows(df_locode_flows, subarea_cleaned, location_type='locode')
ic(len(df_subarea_flows), len(df_locode_flows))
ic(len(flows_subarea), len(flows_locode))

# For subarea-based locations:
locations_subarea = merge_location_flows(df_subarea_locations, subarea_cleaned, location_type='subarea')
# For locode-based locations:
locations_locode = merge_location_flows(df_locode_locations, subarea_cleaned, location_type='locode')
ic(len(df_subarea_locations), len(df_locode_locations))
ic(len(locations_subarea), len(locations_locode))

# For subarea-based unused_actions:
unused_actions_subarea = merge_location_flows(df_subarea_unused_actions, subarea_cleaned, location_type='subarea')
# For locode-based unused_actions:
unused_actions_locode = merge_location_flows(df_locode_unused_actions, subarea_cleaned, location_type='locode')
ic(len(df_subarea_unused_actions), len(df_locode_unused_actions))
ic(len(unused_actions_subarea), len(unused_actions_locode))

ic| len(df_subarea_flows): 5914, len(df_locode_flows): 6579
ic| len(flows_subarea): 5914, len(flows_locode): 6579
ic| len(df_subarea_locations): 376, len(df_locode_locations): 552
ic| len(locations_subarea): 376, len(locations_locode): 552
ic| len(df_subarea_unused_actions): 16391
    len(df_locode_unused_actions): 17654
ic| len(unused_actions_subarea): 16391
    len(unused_actions_locode): 17654


(16391, 17654)

#### 4.2. Save data to output for safekeeping

In [41]:
# #flows
# flows_subarea.to_csv('flows_subarea.csv', index=False)
# flows_locode.to_csv('flows_locode.csv', index=False)

# #locations
# locations_subarea.to_csv('locations_subarea.csv', index=False)
# locations_locode.to_csv('locations_locode.csv', index=False)

# #conversion_rates
# df_subarea_conversion_rates.to_csv('conversion_rates_subarea.csv', index=False)
# df_locode_conversion_rates.to_csv('conversion_rates_locode.csv', index=False)

# #unused_actions
# unused_actions_subarea.to_csv('unused_actions_subarea.csv', index=False)
# unused_actions_locode.to_csv('unused_actions_locode.csv', index=False)

# #metrics
# with open('metrics_subarea.json', 'w', encoding='utf-8') as f:
#     json.dump(dict_subarea_metrics, f, indent=2)

# with open('metrics_locode.json', 'w', encoding='utf-8') as f:
#     json.dump(dict_locode_metrics, f, indent=2)

#### 4.3 part 2

In [47]:
#function for detecting circular movements
def detect_circular_movements(df: pd.DataFrame, min_cycle_length=3, max_cycle_length=5, window_weeks=4) -> pd.DataFrame:
    """
    Detect circular container movements using NetworkX within a given time window.

    This function analyzes movements per equipment type and detects cycles in the movement graph.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing movement data.
    - min_cycle_length (int): Minimum unique locations required to form a valid cycle.
    - max_cycle_length (int): Maximum unique locations allowed in a cycle.
    - window_weeks (int): Time window (in weeks) within which cycles are analyzed.

    Returns:
    - pd.DataFrame: A DataFrame with detected circular movements and their details.
    """

    # Keep only movements with status 'BK' (Booked)
    df = df[df['status'] == 'BK'].copy()

    # List to store detected cycle details
    detailed_cycles = []

    # Process each equipment type separately
    print("Analyzing by equipment type...")
    for eq_type in tqdm(df['eq_type'].unique()):  # Iterate over each unique equipment type
        eq_df = df[df['eq_type'] == eq_type]  # Filter dataset for this equipment type

        # Analyze movements within each time window
        for start_week in eq_df['start_yearweek'].unique():
            end_week = start_week + window_weeks  # Define the end of the window

            # Select movements that occur within the defined time window
            window_df = eq_df[
                (eq_df['start_yearweek'] >= start_week) & 
                (eq_df['start_yearweek'] < end_week)
            ]

            # Skip processing if the window has too few movements to form a cycle
            if len(window_df) < min_cycle_length:
                continue

            # Create a directed graph where nodes are locations and edges represent movements
            G = nx.DiGraph()

            # Add edges (movements) to the graph
            for _, row in window_df.iterrows():
                G.add_edge(
                    row['locode_from'],  # Starting location
                    row['locode_to'],    # Destination location
                    id=row['id_number'],  # Movement ID
                    week=row['start_yearweek'],  # Start week of movement
                    volume=row['volume_teu_proposal'],  # Movement volume
                    mot=row['mot']  # Mode of transport
                )

            # Detect cycles in the graph
            try:
                cycles = list(nx.simple_cycles(G))  # Extract all cycles
                
                # Process each detected cycle
                for cycle in cycles:
                    # Ensure cycle length is within valid range
                    if min_cycle_length <= len(set(cycle)) <= max_cycle_length:
                        cycle_moves = []  # Stores movement details forming the cycle
                        total_volume = 0  # Tracks total volume of cycle movements
                        movement_ids = []  # Stores movement IDs
                        transport_modes = []  # Tracks transport modes

                        valid_cycle = True  # Flag to check if the cycle is complete

                        # Verify each movement exists in the dataset
                        for i in range(len(cycle)):
                            from_loc = cycle[i]  # Current location
                            to_loc = cycle[(i+1) % len(cycle)]  # Next location in cycle

                            # Find movements matching this step in the cycle
                            moves = window_df[
                                (window_df['locode_from'] == from_loc) &
                                (window_df['locode_to'] == to_loc)
                            ]

                            # If any step in the cycle is missing, discard this cycle
                            if moves.empty:
                                valid_cycle = False
                                break

                            # Aggregate movement details
                            total_volume += moves['volume_teu_proposal'].sum()
                            movement_ids.extend(moves['id_number'].tolist())
                            transport_modes.extend(moves['mot'].tolist())
                            cycle_moves.extend(moves.to_dict('records'))

                        # Save valid cycle details
                        if valid_cycle:
                            detailed_cycles.append({
                                'cycle_path': ' -> '.join(cycle + [cycle[0]]),  # Full cycle path
                                'eq_type': eq_type,  # Equipment type
                                'start_yearweek': start_week,  # Start week
                                'total_volume': total_volume,  # Total TEU volume in cycle
                                'movement_count': len(cycle_moves),  # Number of movements in cycle
                                'movement_ids': movement_ids,  # List of movement IDs
                                'transport_modes': transport_modes,  # Transport modes used
                                'locations': cycle  # List of locations forming the cycle
                            })

            except nx.NetworkXError:
                # If graph analysis fails, skip this window
                continue

    # Convert detected cycle list into a DataFrame and return
    return pd.DataFrame(detailed_cycles)

def visualize_movement_graph(df, top_n=100):
    """
    Generate a visual representation of container movements.

    This function creates a directed graph where nodes represent locations and edges represent container movements.

    Parameters:
    - df (pd.DataFrame): DataFrame containing movement data.
    - top_n (int): Number of most frequent locations to visualize.

    Saves:
    - A PNG file ('movement_network.png') showing the network visualization.
    """

    # Create a directed graph
    G = nx.DiGraph()

    # Compute movement frequency per location
    location_freq = pd.concat([
        df['locode_from'].value_counts(), 
        df['locode_to'].value_counts()
    ]).groupby(level=0).sum()

    # Select the top N most active locations
    top_locations = set(location_freq.nlargest(top_n).index)

    # Add edges for selected locations
    print("Building network graph...")
    for _, row in tqdm(df[
        df['locode_from'].isin(top_locations) & 
        df['locode_to'].isin(top_locations)
    ].iterrows()):
        G.add_edge(row['locode_from'], row['locode_to'], 
                   weight=row['volume_teu_proposal'])

    # Generate visualization
    plt.figure(figsize=(20, 20))
    pos = nx.spring_layout(G, k=0.5, iterations=50)

    # Draw nodes (locations)
    nx.draw_networkx_nodes(G, pos, node_size=100, node_color='lightblue')

    # Draw edges (movements)
    nx.draw_networkx_edges(G, pos, edge_color='gray', arrows=True, width=0.5, alpha=0.6)

    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=8)

    # Save graph visualization
    plt.title("Container Movement Network")
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('movement_network.png', dpi=300)
    plt.close()

def circles(df, sample_size=None):
    """
    Main function to execute circular movement detection and visualization.

    Parameters:
    - df (pd.DataFrame): The dataset containing movement records.
    - sample_size (int, optional): Number of rows to sample from the dataset for faster processing.

    Returns:
    - pd.DataFrame: A DataFrame containing detected circular movements.
    """

    # Load dataset
    print("Loading data...")
    df = df.copy()

    # Sample dataset if required
    if sample_size:
        df = df.sample(n=sample_size, random_state=42)

    # Detect circular movements
    print(f"\nAnalyzing {len(df)} movements...")
    circular_movements = detect_circular_movements(df)

    # Display and save results
    print("\nCircular Movement Analysis:")
    print(f"Total circular movements found: {len(circular_movements)}")

    if not circular_movements.empty:
        print("\nSample Circular Movements:")
        print(circular_movements.head())

        print("\nCircular Movements by Equipment Type:")
        print(circular_movements.groupby('eq_type').size())

        # Save detected cycles
        print("output saved as csv")
        circular_movements.to_csv('networkx_circular_movements.csv', index=False)

        # Generate visualization
        visualize_movement_graph(df)
    else:
        print("No circular movements found.")

    return circular_movements


In [48]:
cleaned_data = pd.read_csv('cleaned_hplld_data.csv')
# detect circular movements in booked container routes:
circular_movements = circles(cleaned_data)

Loading data...

Analyzing 242097 movements...
Analyzing by equipment type...


100%|██████████| 15/15 [00:45<00:00,  3.01s/it]



Circular Movement Analysis:
Total circular movements found: 3198

Sample Circular Movements:
                                  cycle_path eq_type  start_yearweek  \
0           USPDX -> USTIW -> USSEA -> USPDX    20GE          202401   
1           USPDX -> USSEA -> USTIW -> USPDX    20GE          202401   
2  CATOR -> CAMTR -> CAEDM -> CAVAN -> CATOR    20GE          202401   
3  CATOR -> CAMTR -> USMES -> CAVAN -> CATOR    20GE          202401   
4  CATOR -> CAMTR -> USDET -> CAVAN -> CATOR    20GE          202401   

   total_volume  movement_count  \
0             0               8   
1             0               7   
2             0              19   
3             0              17   
4             0              17   

                                        movement_ids  \
0  [7804779, 7827912, 7829279, 7830749, 7830009, ...   
1  [7807841, 7850927, 7851492, 7808927, 7830022, ...   
2  [7790652, 7791480, 7792319, 7792320, 7792321, ...   
3  [7790652, 7791480, 7792319, 7792320

91745it [00:04, 21041.95it/s]


In [49]:
#function for detecting inefficient routing
def yearweek_to_date(yearweek: int) -> pd.Timestamp:
    """
    Convert a yearweek integer (YYYYWW) into a Pandas Timestamp representing the start of that week.

    Parameters:
    - yearweek (int): The yearweek value (e.g., 202401 for the first week of 2024).

    Returns:
    - pd.Timestamp: The corresponding date (Monday of that week).
    """
    year = int(str(yearweek)[:4])  # Extract the year (first 4 digits)
    week = int(str(yearweek)[4:])  # Extract the week number (last 2 digits)
    return pd.to_datetime(f"{year}-W{week:02d}-1", format="%Y-W%W-%w")  # Convert to Monday of that week

def detect_routing_inefficiencies_optimized(df: pd.DataFrame, window_weeks=4) -> pd.DataFrame:
    """
    Detect inefficient routing patterns in container movements using graph analysis.

    Parameters:
    - df (pd.DataFrame): The dataset containing movement records.
    - window_weeks (int): The time window (in weeks) used to analyze routes.

    Returns:
    - pd.DataFrame: A DataFrame with identified inefficient routes.
    """

    # Filter out only booked movements
    df = df[df['status'] == 'BK'].copy()

    # List to store inefficient route details
    inefficient_routes = []

    # Process movements per equipment type
    print("Analyzing by equipment type...")
    for eq_type in tqdm(df['eq_type'].unique()):  # Iterate over each unique equipment type
        eq_df = df[df['eq_type'] == eq_type]  # Filter dataset for the current equipment type

        # Analyze movements within each time window
        for start_week in eq_df['start_yearweek'].unique():
            end_week = start_week + window_weeks  # Define the end of the window

            # Select movements that occur within the defined time window
            window_df = eq_df[
                (eq_df['start_yearweek'] >= start_week) & 
                (eq_df['start_yearweek'] < end_week)
            ]

            # Skip analysis if there are not enough movements
            if len(window_df) < 2:
                continue

            # Create a directed graph where nodes are locations and edges represent movements
            G = nx.DiGraph()

            # Aggregate movement details for the given time window
            actual_routes = window_df.groupby(['locode_from', 'locode_to']).agg({
                'volume_teu_proposal': 'sum',  # Sum total TEU volume per route
                'id_number': list,  # Collect movement IDs
                'mot': list,  # Collect modes of transport
                'start_yearweek': list  # Collect weeks of movements
            }).reset_index()

            # Add edges (movements) to the graph
            for _, row in actual_routes.iterrows():
                G.add_edge(
                    row['locode_from'],  # Starting location
                    row['locode_to'],    # Destination location
                    volume=row['volume_teu_proposal']  # Movement volume
                )

            # Check each actual route for inefficiencies
            for _, route in actual_routes.iterrows():
                source = route['locode_from']
                target = route['locode_to']

                try:
                    # Find the shortest path between the source and target
                    shortest_path = nx.shortest_path(G, source, target)
                    shortest_len = len(shortest_path)

                    # Define a cutoff for what is considered an inefficient route
                    cutoff = min(shortest_len * 2, shortest_len + 5)

                    # Find all paths within the cutoff limit
                    actual_paths = list(nx.all_simple_paths(G, source, target, cutoff=cutoff))

                    # Evaluate each actual path for inefficiencies
                    for path in actual_paths:
                        if len(path) > shortest_len:  # The path is inefficient if longer than the shortest path
                            path_moves = []  # List to store movement details for this path
                            path_volume = float('inf')  # Minimum volume constraint

                            # Check if all steps in the path exist in the dataset
                            for i in range(len(path) - 1):
                                moves = window_df[
                                    (window_df['locode_from'] == path[i]) &
                                    (window_df['locode_to'] == path[i+1])
                                ]
                                if moves.empty:
                                    path_volume = 0  # If a segment is missing, discard the path
                                    break
                                path_moves.extend(moves.to_dict('records'))
                                path_volume = min(path_volume, moves['volume_teu_proposal'].sum())

                            # Save inefficient path details if valid
                            if path_volume > 0:
                                inefficient_routes.append({
                                    'eq_type': eq_type,
                                    'start_yearweek': start_week,
                                    'start_date': yearweek_to_date(start_week),
                                    'from_location': source,
                                    'to_location': target,
                                    'optimal_path': ' -> '.join(shortest_path),
                                    'actual_path': ' -> '.join(path),
                                    'volume': path_volume,
                                    'movement_ids': route['id_number'],
                                    'transport_modes': route['mot'],
                                    'weeks': route['start_yearweek'],
                                    'extra_stops': len(path) - shortest_len
                                })

                except nx.NetworkXNoPath:
                    continue  # Skip if no path exists

    return pd.DataFrame(inefficient_routes)

def visualize_movement_graph(df, top_n=100):
    """
    Generate a visual representation of container movements as a network graph.

    Parameters:
    - df (pd.DataFrame): The dataset containing movement records.
    - top_n (int): The number of most frequent locations to visualize.

    Saves:
    - A PNG file ('movement_network.png') showing the network visualization.
    """

    # Create a directed graph
    G = nx.DiGraph()

    # Compute movement frequency per location
    location_freq = pd.concat([
        df['locode_from'].value_counts(), 
        df['locode_to'].value_counts()
    ]).groupby(level=0).sum()

    # Select the top N most active locations
    top_locations = set(location_freq.nlargest(top_n).index)

    # Add edges for selected locations
    print("Building network graph...")
    for _, row in tqdm(df[
        df['locode_from'].isin(top_locations) & 
        df['locode_to'].isin(top_locations)
    ].iterrows()):
        G.add_edge(row['locode_from'], row['locode_to'], 
                   weight=row['volume_teu_proposal'])

    # Generate visualization
    plt.figure(figsize=(20, 20))
    pos = nx.spring_layout(G, k=0.5, iterations=50)

    # Draw nodes (locations)
    nx.draw_networkx_nodes(G, pos, node_size=100, node_color='lightblue')

    # Draw edges (movements)
    nx.draw_networkx_edges(G, pos, edge_color='gray', arrows=True, width=0.5, alpha=0.6)

    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=8)

    # Save graph visualization
    plt.title("Container Movement Network")
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('movement_network.png', dpi=300)
    plt.close()

def inefficient(df):
    """
    Main function to analyze routing inefficiencies in container movements.

    Parameters:
    - df (pd.DataFrame): The dataset containing movement records.

    Returns:
    - pd.DataFrame: A DataFrame with detected routing inefficiencies.
    """

    print("Starting analysis...")
    print(f"Total movements: {len(df)}")

    # Count unique routes
    total_routes = df.groupby(['locode_from', 'locode_to']).size().shape[0]
    print(f"Total unique routes: {total_routes}")

    # Detect inefficient routes
    print("\nAnalyzing inefficient routes...")
    inefficient_routes = detect_routing_inefficiencies_optimized(df)

    # Display results
    print("\nRouting Inefficiency Analysis:")
    print(f"Total inefficient routes found: {len(inefficient_routes)}")

    if not inefficient_routes.empty:
        print("\nSample Inefficient Routes:")
        print(inefficient_routes.head())

        # Save results
        print("output saved as csv")
        inefficient_routes.to_csv('routing_inefficiencies.csv', index=False)

        # Generate visualization
        print("\nCreating network visualization...")
        visualize_movement_graph(df)

    return inefficient_routes


In [50]:
inefficient_routes = inefficient(cleaned_data)

Starting analysis...
Total movements: 242097
Total unique routes: 10202

Analyzing inefficient routes...
Analyzing by equipment type...


100%|██████████| 15/15 [09:00<00:00, 36.03s/it]



Routing Inefficiency Analysis:
Total inefficient routes found: 4612

Sample Inefficient Routes:
  eq_type  start_yearweek start_date from_location to_location  \
0    20GE          202401 2024-01-01         CLSAI       CLVAP   
1    20GE          202401 2024-01-01         CLSAI       CLVAP   
2    20GE          202401 2024-01-01         CLSAI       CNSHK   
3    20GE          202401 2024-01-01         CNHUA       CNGOM   
4    20GE          202401 2024-01-01         CNHUA       CNJMN   

     optimal_path                       actual_path  volume  \
0  CLSAI -> CLVAP           CLSAI -> CNSHK -> CLVAP       5   
1  CLSAI -> CLVAP  CLSAI -> KRPUS -> CNSHK -> CLVAP       5   
2  CLSAI -> CNSHK           CLSAI -> KRPUS -> CNSHK       6   
3  CNHUA -> CNGOM  CNHUA -> CNZQG -> HKHKG -> CNGOM       8   
4  CNHUA -> CNJMN  CNHUA -> CNZQG -> CNSHK -> CNJMN       2   

                                        movement_ids  \
0  [7801282, 7809455, 7830002, 7830024, 7830073, ...   
1  [7801282, 78

91745it [00:04, 20811.74it/s]
